# Calculating with functions

We are used to performing numerical calculations with numbers, using floating-point approximations of reals  where the approximation differs by (at most) machine epsilon from the exact value.

In this lecture we will see that this idea can be extended to do **calculations with functions**, using **approximations of functions** that differ from the exact function by around machine epsilon! 

The basic idea is a simple one: out of all functions, there are a few that we understand well and are able to calculate efficiently with, namely polynomials. So we will use in our manipulations not the original functions, but rather **polynomial approximations** that are machine epsilon-close. With careful choices of method and algorithm, it turns out that this may be done very efficiently, using Chebyshev interpolation and approximation by Chebyshev polynomials.

These ideas were first developed by [Nick Trefethen](https://people.maths.ox.ac.uk/trefethen/) and his group in the [Chebfun](http://www.chebfun.org/) package for Matlab. An implementation in Julia is available in the 
[ApproxFun.jl](https://github.com/JuliaApproximation/ApproxFun.jl) package.

## Naive polynomial interpolation

One way to approximate data is by interpolation. Given $n+1$ data points $(x_i, y_i)_{i=0}^{n}$, there is a unique polynomial $p$ of (maximum) degree $n$ that passes through them. We can find it in a simple (but naive, and not efficient) way as follows:

Let $$p(x) = a_0 + a_1 \, x + a_2 \, x^2 + \cdots + a_n \, x^n$$.

Then we have $n+1$ simultaneous equations 
$$p(x_i) = a_0 + a_1 \, x_i + a_2 \, x_i^2 + \cdots + a_n \, x_i^n = y_i$$
to solve, i.e.

$$
\begin{pmatrix}
1 & x_0 & x_0^2 & \cdots & x_0^n \\
1 & x_1 & x_1^2 & \cdots & x_1^n \\
1 & x_2 & x_2^2 & \cdots & x_2^n \\
\vdots
\\ 
1 & x_n & x_n^2 & \cdots & x_n^n \\
\end{pmatrix}
\cdot 
\begin{pmatrix}
a_0 \\ \vdots \\ a_n
\end{pmatrix}
=
\begin{pmatrix}
y_0 \\ \vdots \\ y_n
\end{pmatrix}
$$

The matrix above is known as a Vandermonde matrix. The code below generalizes to allow for rectangular "Vandermonde" matrices, but defaults to the square case.

In [1]:
versioninfo()

Julia Version 1.0.2
Commit d789231e99 (2018-11-08 20:11 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i7-6600U CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.0 (ORCJIT, skylake)
Environment:
  JULIA_EDITOR = vim


In [2]:
vander(x,n=length(x)-1) = x.^(0:n)'

vander (generic function with 2 methods)

In [3]:
vander(3:5,2)

3×3 Array{Int64,2}:
 1  3   9
 1  4  16
 1  5  25

Given a function $f$, we can choose points $x_i$ and let $y_i = f(x_i)$. It is natural (but, it turns out, incorrect) to try equi-spaced points:

In [4]:
lo, hi = -1, 1  # the interval we work over

(-1, 1)

In [5]:
n = 10
x = range(lo, hi, length = n+1)
M = vander(x)

11×11 Array{Float64,2}:
 1.0  -1.0  1.0   -1.0    1.0     -1.0      …  -1.0          1.0        
 1.0  -0.8  0.64  -0.512  0.4096  -0.32768     -0.134218     0.107374   
 1.0  -0.6  0.36  -0.216  0.1296  -0.07776     -0.0100777    0.00604662 
 1.0  -0.4  0.16  -0.064  0.0256  -0.01024     -0.000262144  0.000104858
 1.0  -0.2  0.04  -0.008  0.0016  -0.00032     -5.12e-7      1.024e-7   
 1.0   0.0  0.0    0.0    0.0      0.0      …   0.0          0.0        
 1.0   0.2  0.04   0.008  0.0016   0.00032      5.12e-7      1.024e-7   
 1.0   0.4  0.16   0.064  0.0256   0.01024      0.000262144  0.000104858
 1.0   0.6  0.36   0.216  0.1296   0.07776      0.0100777    0.00604662 
 1.0   0.8  0.64   0.512  0.4096   0.32768      0.134218     0.107374   
 1.0   1.0  1.0    1.0    1.0      1.0      …   1.0          1.0        

In [6]:
a = M \ exp.(x)   # kind of looks like taylor series, almost, but not exactly 

11-element Array{Float64,1}:
 0.9999999999999999    
 1.0000000000374658    
 0.5000000000031157    
 0.16666666529603313   
 0.0416666665526852    
 0.008333345756140873  
 0.0013888899220864414 
 0.00019837119018122377
 2.47981343937471e-5   
 2.81136398031263e-6   
 2.802029626550234e-7  

The entries of `a` are the coefficients of the fitted polynomial. We can now build (again, naively) the interpolating polynomial:

In [7]:
import Base: +, *, -

*(α::Number, f::Function) = x -> α*f(x)
+(f::Function, g::Function) = x -> f(x) + g(x)
-(f::Function, g::Function) = x -> f(x) - g(x)

- (generic function with 176 methods)

In [8]:
pp = sum(a[i]*(x->x^(i-1)) for i in 1:length(a))
f(k) = sum((1/factorial(i))*(x->x^i) for i in k:-1:0)

f (generic function with 1 method)

In [9]:
using Plots, Interact
#PyPlot.svg(true)
gr()

HTML{String}("    <script id=\"webio-setup-16289660938098645807\">\n    // Immediately-invoked-function-expression to avoid global variables.\n    (function() {\n        var warning_div = document.getElementById(\"webio-warning-16319215659088452174\");\n        var hide = function () {\n            var script = document.getElementById(\"webio-setup-16289660938098645807\");\n            var parent = script && script.parentElement;\n            var grandparent = parent && parent.parentElement;\n            if (grandparent) {\n                grandparent.style.display = \"none\";\n            }\n            warning_div.style.display = \"none\";\n        };\n        if (window.require && require.defined) {\n            // Jupyter notebook.\n            if (require.defined(\"nbextensions/webio/main\")\n                    || require.defined(\"/assetserver/8f4ac2f32dd6025e083cecadc13ddc89525c7ec1-dist/main.js\")) {\n                // Extension already loaded.\n                hide();\n                return;\n            }\n            console.warn(\"Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).\");\n            require([\"/assetserver/8f4ac2f32dd6025e083cecadc13ddc89525c7ec1-dist/main.js\"], function (webIOModule) {\n                webIOModule.load_ipython_extension();\n            });\n            warning_div.innerHTML = \"<strong>Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).</strong>\";\n        } else if (window.location.pathname.includes(\"/lab\")) {\n            // Guessing JupyterLab\n            warning_div.innerHTML = \"WebIO does not support JupyterLab yet.\";\n        }\n    })();\n    </script>\n    <div\n        id=\"webio-warning-16319215659088452174\"\n        class=\"output_text output_stderr\"\n        style=\"padding: 1em; font-weight: bold;\"\n    >\n        Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n        <!-- TODO: link to installation docs. -->\n    </div>\n")

Plots.GRBackend()

In [10]:
#l = @layout [a b]
#p = plot(f, lo, hi, layout=l,label="function")
#@manipulate for k=2:100

plot(pp, lo, hi, label="interpolant")
plot( x-> log10(abs((f-pp)(x))), lo, hi, label="error of the interpolant")
#plot!( x-> log10(abs((f-ttk(10))(x))), lo, hi, label="error in taylor")
#end

ArgumentError: ArgumentError: reducing over an empty collection is not allowed

However, this goes wrong for certain functions:

In [11]:
h(x) = 1 / (1 + 25x^2)  # the Runge function

h (generic function with 1 method)

In [12]:
x

-1.0:0.2:1.0

In [13]:
function interpolate(f, x)
    lo, hi = -1.0, 1.0
    M = vander(x)
    y = f.(x)

    a = M \ y

    p = sum(a[i]*(x->x^(i-1)) for i in 1:length(a))
        
    return p, y, a
end

interpolate (generic function with 1 method)

In [14]:
using Interact

In [40]:
h(x) = 1 / (1 + 25x^2)  # the Runge function

#n = 20
@manipulate for n in 2:20
    x = LinRange(-1, 1, n+1)
    @time p, y = interpolate(h, x)

    plot(x->h(x), lo, hi)
    scatter!(x, y)
    plot!(p, lo, hi, label="Interpolant")
end

Widget{:manipulate,Any}(OrderedDict{Symbol,Any}(:n=>Widget{:slider,Int64}(OrderedDict{Symbol,Any}(:changes=>Observable{Int64}("ob_08", 0, Any[SyncCallback(Scope("knockout-component-f339b53a-fdd6-4bbf-8c8d-58f53dd0a2b0", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["n"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"text-align:right;width:18%")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>20,:min=>2,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: value, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"flex-grow:1; margin: 0 2%")), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: value")), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"width:18%")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"display:flex; justify-content:center; align-items:center;")), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64}(#= circular reference @-6 =#), nothing),"value"=>(Observable{Int64}("ob_07", 11, Any[SyncCallback(Scope(#= circular reference @-6 =#), SyncCallback(Scope(#= circular reference @-7 =#), ##36#38{Scope,String}(Scope(#= circular reference @-8 =#), "value"))), #g#15{##16#17{##67#70,Observable{Any}},Tuple{Observable{Int64}}}(##16#17{##67#70,Observable{Any}}(##67#70(), Observable{Any}("ob_09", Plot{Plots.GRBackend() n=3}, Any[])), (Observable{Int64}(#= circular reference @-4 =#),))]), nothing)), Set(String[]), nothing, Any["knockout"=>"/home/shashi/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/home/shashi/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/home/shashi/.julia/packages/InteractBase/Q4IkI/src/../assets/all.js", "/home/shashi/.julia/packages/InteractBase/Q4IkI/src/../assets/style.css", "/home/shashi/.julia/packages/InteractBulma/Ohu5Y/src/../assets/main.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"changes\\\":0,\\\"value\\\":11}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"changes\"].subscribe((function (val){!(this.valueFromJulia[\"changes\"]) ? (WebIO.setval({\"name\":\"changes\",\"scope\":\"knockout-component-f339b53a-fdd6-4bbf-8c8d-58f53dd0a2b0\",\"id\":\"ob_08\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJulia[\"changes\"]=false}),self),this[\"value\"].

## Chebyshev points

This problem can be **eliminated** by choosing interpolation points which **cluster near the boundary** in a certain way. A simple set are the Chebyshev points, defined by projecting down equi-spaced points on the unit circle to the $x$-axis.

In [41]:
using Plots
gr()

Plots.GRBackend()

In [50]:
@manipulate for n in 2:1:100

#n = 100
    plot(x->sqrt(1-x^2), -1, 1,ylim=(0,1), aspect_ratio=1,label="")
    
    θs = LinRange(0, π, n)
    c,s = cos.(θs), sin.(θs)
    
    for i=1:length(c)     
        plot!([c[i], c[i]], [0, s[i]], ls=:dot, legend=false, label="T$i")
    end
    
    scatter!(c, s, markersize=1,label="")
    scatter!(c, fill(0., length(s)), markersize=1,label="")
end

Widget{:manipulate,Any}(OrderedDict{Symbol,Any}(:n=>Widget{:slider,Int64}(OrderedDict{Symbol,Any}(:changes=>Observable{Int64}("ob_27", 0, Any[SyncCallback(Scope("knockout-component-083fe6af-a3a5-4652-9a72-1fcd52fb361e", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["n"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"text-align:right;width:18%")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>100,:min=>2,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: value, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"flex-grow:1; margin: 0 2%")), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: value")), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"width:18%")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"display:flex; justify-content:center; align-items:center;")), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64}(#= circular reference @-6 =#), nothing),"value"=>(Observable{Int64}("ob_26", 51, Any[SyncCallback(Scope(#= circular reference @-6 =#), SyncCallback(Scope(#= circular reference @-7 =#), ##36#38{Scope,String}(Scope(#= circular reference @-8 =#), "value"))), #g#15{##16#17{##109#112,Observable{Any}},Tuple{Observable{Int64}}}(##16#17{##109#112,Observable{Any}}(##109#112(), Observable{Any}("ob_28", Plot{Plots.GRBackend() n=54}, Any[])), (Observable{Int64}(#= circular reference @-4 =#),))]), nothing)), Set(String[]), nothing, Any["knockout"=>"/home/shashi/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/home/shashi/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/home/shashi/.julia/packages/InteractBase/Q4IkI/src/../assets/all.js", "/home/shashi/.julia/packages/InteractBase/Q4IkI/src/../assets/style.css", "/home/shashi/.julia/packages/InteractBulma/Ohu5Y/src/../assets/main.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"changes\\\":0,\\\"value\\\":51}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"changes\"].subscribe((function (val){!(this.valueFromJulia[\"changes\"]) ? (WebIO.setval({\"name\":\"changes\",\"scope\":\"knockout-component-083fe6af-a3a5-4652-9a72-1fcd52fb361e\",\"id\":\"ob_27\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJulia[\"changes\"]=false}),self),this[\"v

In [53]:
chebyshev_points(n) = cos.(LinRange(0, π, n))

chebyshev_points (generic function with 1 method)

Interpolating in these points is **well-behaved**:

In [54]:
chebyshev_points(11)

11-element Array{Float64,1}:
  1.0                  
  0.9510565162951535   
  0.8090169943749475   
  0.5877852522924731   
  0.30901699437494745  
  6.123233995736766e-17
 -0.30901699437494734  
 -0.587785252292473    
 -0.8090169943749473   
 -0.9510565162951535   
 -1.0                  

In [55]:
using Interact

In [ ]:
h(x) = 1 / (1 + 25x^2)  # the Runge function

@manipulate for n in 2:100
    
#n = 5

    x = chebyshev_points(n)
    p, y = interpolate(h, x)

    plot(h, lo, hi, label="Runge function")
    scatter!(x, y)
    plot!(p, lo, hi)
end

So we see that we should choose the polynomial with which to approximate a function $f$ by **interpolation in Chebyshev points**. The question is how many points we should use?

In [ ]:
interpolate(h, chebyshev_points(500))

In [37]:
pp, yy, aa = interpolate(exp, big.(chebyshev_points(200)));

In [38]:
Float64.(aa)

200-element Array{Float64,1}:
  1.0        
  1.0        
  0.5        
  0.166667   
  0.0416667  
  0.00833333 
  0.00138889 
  0.000198413
  2.48016e-5 
  2.75573e-6 
  2.75573e-7 
  2.50521e-8 
  2.08768e-9 
  ⋮          
  9.74733e-15
 -4.84124e-13
 -1.41423e-15
  5.22069e-14
  1.5065e-16 
 -4.4323e-15 
 -1.13443e-17
  2.7779e-16 
  5.44343e-19
 -1.14268e-17
 -1.25871e-20
  2.31416e-19

## Approximation by Chebyshev polynomials

It turns out that the standard monomial basis ($1$, $x$, $x^2$, ...) is **not** a good basis for polynomial interpolation. A good basis is the set of **Chebyshev polynomials**, defined by

$$T_n(x) = \mathrm{Re}(z^n) = \cos(n \theta) = \cos(n \cos^{-1}(x))$$

In [ ]:
T(n) = x -> cos(n * acos(x))  # a simple, but unintuitive way to define these in Julia

We can do polynomial interpolation using Chebyshev polynomials by writing a similar formula to that above:

$$f(x) = \sum_n \alpha_n T_n(x)$$

and again doing naive interpolation:

In [ ]:
chebyshev_points(n) = cos.(LinRange(0, π, n))

In [ ]:
function chebyshev_interpolate(f, xx)
    lo, hi = -1.0, 1.0
    
    n = length(xx) - 1

    yy = [f(x) for x in xx]

    M = [T(i)(x) for x in xx, i in 0:n]
    

    aa = M \ yy

    pp = sum(aa[i]*(x->x^(i-1)) for i in 1:length(aa))
        
    return pp, yy, aa
end

In [ ]:
xx = chebyshev_points(25)
pp, yy, aa = chebyshev_interpolate(exp, xx)

In [ ]:
aa

The point is that the coefficients decay quickly (until they reach something of order machine epsilon) for smooth functions:

In [ ]:
scatter(abs.(aa), yscale=:log10)

In [ ]:
n = 11
xx = chebyshev_points(n)
ff = exp.(xx)

In [ ]:
real(fft(ff)) / (n/2)

The idea is thus to adaptively choose the number of points at which the function is sampled, until the tail of the  coefficients in the Chebyshev expansion have decayed to machine precision.

## From data to Chebyshev coefficients

It turns out that there is a key relationship between Chebyshev points and Chebyshev polynomials, which enables us to find the coefficients $\alpha_n$ in the expansion in Chebyshev polynomials in time $\mathcal{O}(n \, \log n)$, instead of the $\mathcal{O}(n^3)$ that the naive matrix formulation requires. This entails the use of the discrete cosine transform, a close relative of the FFT.

The idea is as follows. Suppose that we in fact know the expansion $f = \sum a_n T_n$. Then evaluating this in a Chebyshev point $x_i$ gives

$$f(x_i) = \sum a_k T_k(x_i)$$.

But $x_i = \cos (i \pi / n)$ and $T_k(x) = \cos(k \cos^{-1}(x))$, so

$$f(x_j) = \sum_k a_k \cos(j \, k \pi / n)$$.

That is, the data $f(x_i)$ are given by the **discrete cosine transform** of the coefficients $a_n$. Furthermore, from this we see that we can calculate the $a_n$ by taking the inverse discrete cosine transform of the data. This is very closely related to the FFT and can be similarly calculated in $\mathcal{O}(n \, \log n)$ time:

$$\sum_j f(x_j) \cos(j l\pi/n) = \sum_j \sum_k a_k \cos(j k\pi / n) \cos(j l\pi/n)$$

$$ = \sum_k a_k (n/2) \delta_{kl} = (n/2) a_l$$

In [ ]:
ff

In [ ]:
fft(ff)

In [ ]:
?fft

Discrete cosine transform matrix:

In [ ]:
n = 11
xx = chebyshev_points(n+1)
M = [T(i)(x) for x in xx, i in 0:n]


Inverse discrete cosine transform matrix:

In [ ]:
(n/2)*inv(M)

## Barycentric interpolation

One final piece of the puzzle is how to evaluate the interpolating function at different points $x$. We could construct the whole function as the sum $a_n T_n$, but this is not actually necessary. Instead we use barycentric interpolation [Trefethen 2004]: given data $f_j$ at nodes $x_j$, the value of the interpolating polynomial at $x$ is

$$p(x) = \frac{\sum_j \frac{w_j}{x-x_j} f_j}{\sum_j \frac{w_j}{x - x_j}}$$

The weights are given by 

$$w_j = \frac{1}{\prod_{j \neq k} (x_j - x_k)}$$.

These depend **only** on the interpolation points, not on the data. For Chebyshev points, they are

$$w_j = (-1)^j \delta_j,$$

where $\delta_j = 1/2$ for $j=0$ and $j=n$, and $1$ otherwise.

In [ ]:
function ww(j)
    j -= 1  # 0-based
    sign = iseven(j) ? 1 : -1
    value = (j==0 || j==n) ? 0.5 : 1.0
    
    return sign*value
end

In [ ]:
n = 11
xx = chebyshev_points(n)
g(x) = 1 / (1 + 25x^2)
ff = g.(xx)

In [ ]:
using Plots

In [ ]:
ppp(x) = sum(ww(j) / (x-xx[j]) * ff[j] for j = 1:length(xx)) / sum(ww(j) / (x-xx[j]) for j = 1:length(xx))

In [ ]:
using Interact

In [ ]:
g(x) = 1 / (1 + 25x^2)
n = 15
#@manipulate for n in 2:100
    xx = chebyshev_points(n)
   
    ff = g.(xx)

    plot(x->ppp(x), -1, 1)
    plot!(f, -1, 1)
#end

In [ ]:
@manipulate for i in 1:10
    i^2
end

In [ ]:
using PyCall

In [ ]:
Pkg.add("PyPlot")
Pkg.add("PlotlyJS")

In [ ]:
ENV["PYTHON"]=""
ENV["JUPYTER"] = ""

Pkg.build("PyCall")
Pkg.build("PyPlot")
Pkg.build("IJulia")

In [ ]:
Pkg.build("PyCall")

In [ ]:
Pkg.build("Interact")

In [ ]:
Pkg.build("IJulia")

In [ ]:
using Interact

In [ ]:
@manipulate for i in 1:10
    i^2
end

## ApproxFun

In [ ]:
using ApproxFun

In [ ]:
Fun

In [ ]:
x = Fun()

In [ ]:
# 0*T_0 + 1.0*T_1 = 0 + x = x

In [ ]:
x(0.3)

In [ ]:
x(1.5)

In [ ]:
x^2

In [ ]:
# 0.5 T_0 + 0.5*T_2 = 0.5 + 0.5*(2x^2 - 1) = x^2

In [ ]:
f = sin(x^2)

In [ ]:
f.coefficients

In [ ]:
using Plots

In [ ]:
plot(f, -1, 1)

In [ ]:
f

In [ ]:
f'